# 🌟 Sun Colab AI Processing Server

**Notebook URL**: https://colab.research.google.com/drive/1EwfBj0nC9St-2hB1bv2zGWWDTcS4slsx

**Secret Key**: `sun_colab`

This notebook provides GPU-accelerated AI processing for your platform with enhanced security using Colab secrets.

## 🔐 Security Setup

First, let's set up secure access using Colab secrets:

In [ ]:
# Setup Colab secrets and authentication
from google.colab import userdata
import os
import json

print("🔐 Setting up secure authentication...")

# Get the sun_colab secret key
try:
    sun_colab_key = userdata.get('sun_colab')
    print("✅ Successfully retrieved sun_colab secret")
    
    # Set as environment variable for the session
    os.environ['SUN_COLAB_KEY'] = sun_colab_key
    
except Exception as e:
    print(f"❌ Failed to get sun_colab secret: {e}")
    print("\n📝 To set up the secret:")
    print("1. Click the 🔑 key icon in the left sidebar")
    print("2. Add a new secret named 'sun_colab'")
    print("3. Set a secure API key value")
    print("4. Re-run this cell")
    
    # Generate a temporary key for demonstration
    import secrets
    temp_key = f"sun_colab_{secrets.token_hex(16)}"
    os.environ['SUN_COLAB_KEY'] = temp_key
    print(f"\n⚠️  Using temporary key: {temp_key[:20]}...")

# Verify GPU access
import torch
print(f"\n🚀 GPU Status: {'✅ Available' if torch.cuda.is_available() else '❌ Not Available'}")
if torch.cuda.is_available():
    print(f"   Device: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 📦 Install Dependencies

In [ ]:
# Install required packages with optimized versions
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers[torch] accelerate
!pip install -q diffusers==0.21.4
!pip install -q sentence-transformers
!pip install -q fastapi uvicorn[standard]
!pip install -q pyngrok
!pip install -q xformers
!pip install -q opencv-python-headless
!pip install -q pillow==9.5.0
!pip install -q requests aiohttp
!pip install -q psutil GPUtil

print("✅ All dependencies installed successfully")

## 🧠 Sun Colab Processing Engine

In [ ]:
import torch
import numpy as np
from typing import Dict, List, Any, Optional, Union
import base64
import io
from PIL import Image
import json
from datetime import datetime
import logging
import asyncio
import time
import hashlib
import os

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class SunColabProcessor:
    """Main processing engine for Sun Colab integration"""
    
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.api_key = os.environ.get('SUN_COLAB_KEY')
        self.session_id = hashlib.md5(f"sun_colab_{datetime.now()}".encode()).hexdigest()[:8]
        
        logger.info(f"🌟 Sun Colab Processor initialized")
        logger.info(f"   Device: {self.device}")
        logger.info(f"   Session ID: {self.session_id}")
        logger.info(f"   API Key: {'✅ Set' if self.api_key else '❌ Missing'}")
        
        # Model cache
        self.models = {}
        self.stats = {
            'session_start': datetime.now(),
            'tasks_processed': 0,
            'total_processing_time': 0,
            'models_loaded': 0,
            'errors': 0
        }
        
        # Initialize core models
        self.text_model = None
        self.image_model = None
        self.embedding_model = None
        
    def authenticate_request(self, request_key: str) -> bool:
        """Authenticate incoming requests"""
        return request_key == self.api_key
    
    async def load_text_model(self, model_name: str = "microsoft/DialoGPT-medium"):
        """Load text generation model with optimization"""
        if self.text_model is None:
            logger.info(f"🔤 Loading text model: {model_name}")
            start_time = time.time()
            
            from transformers import AutoModelForCausalLM, AutoTokenizer
            
            try:
                self.text_tokenizer = AutoTokenizer.from_pretrained(model_name)
                self.text_model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    torch_dtype=torch.float16 if self.device.type == 'cuda' else torch.float32,
                    low_cpu_mem_usage=True
                )
                self.text_model.to(self.device)
                self.text_model.eval()
                
                # Add padding token
                if self.text_tokenizer.pad_token is None:
                    self.text_tokenizer.pad_token = self.text_tokenizer.eos_token
                
                load_time = time.time() - start_time
                self.stats['models_loaded'] += 1
                logger.info(f"✅ Text model loaded in {load_time:.2f}s")
                
            except Exception as e:
                logger.error(f"❌ Failed to load text model: {e}")
                self.stats['errors'] += 1
                raise
    
    async def load_image_model(self, model_id: str = "runwayml/stable-diffusion-v1-5"):
        """Load image generation model with memory optimization"""
        if self.image_model is None:
            logger.info(f"🎨 Loading image model: {model_id}")
            start_time = time.time()
            
            from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
            
            try:
                self.image_model = StableDiffusionPipeline.from_pretrained(
                    model_id,
                    torch_dtype=torch.float16 if self.device.type == 'cuda' else torch.float32,
                    safety_checker=None,
                    requires_safety_checker=False
                )
                
                # Optimize scheduler
                self.image_model.scheduler = DPMSolverMultistepScheduler.from_config(
                    self.image_model.scheduler.config
                )
                
                self.image_model = self.image_model.to(self.device)
                
                # Enable memory efficient attention
                if hasattr(self.image_model, 'enable_xformers_memory_efficient_attention'):
                    try:
                        self.image_model.enable_xformers_memory_efficient_attention()
                        logger.info("   ⚡ xformers memory optimization enabled")
                    except:
                        logger.info("   ⚠️  xformers not available, using default attention")
                
                # Enable model CPU offload for memory efficiency
                if hasattr(self.image_model, 'enable_model_cpu_offload'):
                    self.image_model.enable_model_cpu_offload()
                    logger.info("   💾 CPU offload enabled for memory efficiency")
                
                load_time = time.time() - start_time
                self.stats['models_loaded'] += 1
                logger.info(f"✅ Image model loaded in {load_time:.2f}s")
                
            except Exception as e:
                logger.error(f"❌ Failed to load image model: {e}")
                self.stats['errors'] += 1
                raise
    
    async def load_embedding_model(self, model_name: str = "all-MiniLM-L6-v2"):
        """Load embedding model"""
        if self.embedding_model is None:
            logger.info(f"🔗 Loading embedding model: {model_name}")
            start_time = time.time()
            
            from sentence_transformers import SentenceTransformer
            
            try:
                self.embedding_model = SentenceTransformer(model_name)
                self.embedding_model.to(self.device)
                
                load_time = time.time() - start_time
                self.stats['models_loaded'] += 1
                logger.info(f"✅ Embedding model loaded in {load_time:.2f}s")
                
            except Exception as e:
                logger.error(f"❌ Failed to load embedding model: {e}")
                self.stats['errors'] += 1
                raise
    
    async def process_text(self, text: str, model: str = "default", 
                          parameters: Optional[Dict] = None) -> Dict[str, Any]:
        """Enhanced text processing with Sun Colab optimizations"""
        start_time = time.time()
        
        try:
            await self.load_text_model()
            
            # Enhanced parameters
            params = parameters or {}
            max_length = min(params.get('max_length', 150), 512)  # Prevent memory issues
            temperature = params.get('temperature', 0.8)
            top_p = params.get('top_p', 0.9)
            do_sample = params.get('do_sample', True)
            
            # Tokenize with proper attention mask
            inputs = self.text_tokenizer(
                text, 
                return_tensors='pt', 
                truncation=True, 
                max_length=256,
                padding=True
            ).to(self.device)
            
            # Generate with optimizations
            with torch.no_grad():
                outputs = self.text_model.generate(
                    inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_length=max_length,
                    temperature=temperature,
                    top_p=top_p,
                    do_sample=do_sample,
                    pad_token_id=self.text_tokenizer.eos_token_id,
                    num_return_sequences=1,
                    early_stopping=True
                )
            
            # Decode only the new tokens
            generated_text = self.text_tokenizer.decode(
                outputs[0][inputs.input_ids.shape[1]:], 
                skip_special_tokens=True
            )
            
            processing_time = time.time() - start_time
            self.stats['tasks_processed'] += 1
            self.stats['total_processing_time'] += processing_time
            
            return {
                'generated_text': text + generated_text,
                'new_text': generated_text,
                'model': model,
                'parameters': params,
                'processing_time': processing_time,
                'session_id': self.session_id
            }
            
        except Exception as e:
            self.stats['errors'] += 1
            logger.error(f"Text processing error: {e}")
            raise
    
    async def generate_image(self, prompt: str, style: Optional[str] = None,
                           size: str = "512x512", **kwargs) -> Dict[str, Any]:
        """Enhanced image generation with Sun Colab optimizations"""
        start_time = time.time()
        
        try:
            await self.load_image_model()
            
            # Parse size and limit for memory
            width, height = map(int, size.split('x'))
            width = min(width, 1024)
            height = min(height, 1024)
            
            # Apply style enhancements
            enhanced_prompt = prompt
            if style:
                style_enhancements = {
                    'anime': ', anime style, manga, cel shading, vibrant colors',
                    'realistic': ', photorealistic, 8k uhd, professional photography, detailed',
                    'artistic': ', oil painting, fine art, masterpiece, detailed brushwork',
                    'cyberpunk': ', cyberpunk style, neon lights, futuristic, high tech',
                    'fantasy': ', fantasy art, magical, ethereal, mystical atmosphere',
                    'portrait': ', portrait photography, professional lighting, detailed face',
                    'landscape': ', landscape photography, wide angle, natural lighting'
                }
                enhanced_prompt += style_enhancements.get(style, '')
            
            # Optimize generation parameters
            num_steps = min(kwargs.get('steps', 25), 50)  # Limit steps for speed
            guidance_scale = kwargs.get('guidance_scale', 7.5)
            
            # Generate with memory management
            with torch.autocast(device_type='cuda' if self.device.type == 'cuda' else 'cpu'):
                image = self.image_model(
                    enhanced_prompt,
                    height=height,
                    width=width,
                    num_inference_steps=num_steps,
                    guidance_scale=guidance_scale,
                    negative_prompt="blurry, low quality, distorted, deformed"
                ).images[0]
            
            # Convert to base64
            buffered = io.BytesIO()
            image.save(buffered, format="PNG", optimize=True)
            img_base64 = base64.b64encode(buffered.getvalue()).decode()
            
            processing_time = time.time() - start_time
            self.stats['tasks_processed'] += 1
            self.stats['total_processing_time'] += processing_time
            
            # Clear GPU cache
            if self.device.type == 'cuda':
                torch.cuda.empty_cache()
            
            return {
                'image': img_base64,
                'prompt': enhanced_prompt,
                'original_prompt': prompt,
                'size': f"{width}x{height}",
                'style': style,
                'steps': num_steps,
                'processing_time': processing_time,
                'session_id': self.session_id
            }
            
        except Exception as e:
            self.stats['errors'] += 1
            logger.error(f"Image generation error: {e}")
            # Clear GPU cache on error
            if self.device.type == 'cuda':
                torch.cuda.empty_cache()
            raise
    
    async def generate_embeddings(self, texts: List[str], 
                                model: str = "sentence-transformers") -> Dict[str, Any]:
        """Batch embedding generation with optimization"""
        start_time = time.time()
        
        try:
            await self.load_embedding_model()
            
            # Batch processing for efficiency
            batch_size = min(len(texts), 32)  # Optimal batch size
            
            embeddings = []
            for i in range(0, len(texts), batch_size):
                batch = texts[i:i + batch_size]
                
                with torch.no_grad():
                    batch_embeddings = self.embedding_model.encode(
                        batch,
                        convert_to_tensor=True,
                        device=self.device,
                        show_progress_bar=False,
                        normalize_embeddings=True
                    )
                    embeddings.append(batch_embeddings.cpu().numpy())
            
            # Concatenate all batches
            final_embeddings = np.vstack(embeddings)
            
            processing_time = time.time() - start_time
            self.stats['tasks_processed'] += 1
            self.stats['total_processing_time'] += processing_time
            
            return {
                'embeddings': final_embeddings.tolist(),
                'shape': final_embeddings.shape,
                'model': model,
                'processing_time': processing_time,
                'session_id': self.session_id
            }
            
        except Exception as e:
            self.stats['errors'] += 1
            logger.error(f"Embedding generation error: {e}")
            raise
    
    def get_system_stats(self) -> Dict[str, Any]:
        """Get comprehensive system statistics"""
        import psutil
        
        # GPU stats
        gpu_stats = {}
        if torch.cuda.is_available():
            gpu_stats = {
                'name': torch.cuda.get_device_name(0),
                'memory_allocated': torch.cuda.memory_allocated(0) / 1e9,
                'memory_total': torch.cuda.get_device_properties(0).total_memory / 1e9,
                'utilization': torch.cuda.utilization() if hasattr(torch.cuda, 'utilization') else 0
            }
        
        # CPU and memory stats
        memory = psutil.virtual_memory()
        
        return {
            'session_info': {
                'session_id': self.session_id,
                'start_time': self.stats['session_start'].isoformat(),
                'uptime_minutes': (datetime.now() - self.stats['session_start']).total_seconds() / 60
            },
            'processing_stats': self.stats,
            'models_loaded': {
                'text': self.text_model is not None,
                'image': self.image_model is not None,
                'embedding': self.embedding_model is not None
            },
            'system_resources': {
                'cpu_percent': psutil.cpu_percent(),
                'memory_percent': memory.percent,
                'memory_available_gb': memory.available / 1e9,
                'gpu': gpu_stats
            },
            'device': str(self.device),
            'authenticated': bool(self.api_key)
        }

# Initialize the Sun Colab processor
processor = SunColabProcessor()
print(f"\n✅ Sun Colab Processor initialized successfully!")
print(f"   Session ID: {processor.session_id}")
print(f"   Device: {processor.device}")

## 🌐 FastAPI Server with Authentication

In [ ]:
from fastapi import FastAPI, HTTPException, Depends, Header
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from pydantic import BaseModel
from typing import List, Optional, Dict, Any
import uvicorn
import nest_asyncio
import threading
import time

# Allow nested asyncio for Colab
nest_asyncio.apply()

# Security
security = HTTPBearer()

# Request models
class ProcessingRequest(BaseModel):
    task_id: str
    task_type: str
    payload: Dict[str, Any]
    priority: int = 5

class TextRequest(BaseModel):
    text: str
    model: str = "default"
    parameters: Optional[Dict[str, Any]] = None

class ImageRequest(BaseModel):
    prompt: str
    style: Optional[str] = None
    size: str = "512x512"
    steps: int = 25
    guidance_scale: float = 7.5

class EmbeddingRequest(BaseModel):
    texts: List[str]
    model: str = "sentence-transformers"

# Authentication
def verify_token(credentials: HTTPAuthorizationCredentials = Depends(security)):
    """Verify the API token"""
    if not processor.authenticate_request(credentials.credentials):
        raise HTTPException(status_code=401, detail="Invalid authentication token")
    return credentials.credentials

# Create FastAPI app
app = FastAPI(
    title="Sun Colab AI Processing Server",
    version="1.0.0",
    description="Secure GPU-accelerated AI processing with Sun Colab"
)

@app.get("/")
async def root():
    """Public endpoint with basic info"""
    return {
        "message": "Sun Colab AI Processing Server",
        "version": "1.0.0",
        "session_id": processor.session_id,
        "capabilities": [
            "text_generation",
            "image_generation", 
            "embeddings",
            "batch_processing"
        ],
        "authentication": "required",
        "gpu_available": torch.cuda.is_available()
    }

@app.get("/health")
async def health_check():
    """Public health check"""
    stats = processor.get_system_stats()
    
    return {
        "status": "healthy",
        "session_id": processor.session_id,
        "uptime_minutes": stats['session_info']['uptime_minutes'],
        "load": stats['system_resources']['cpu_percent'] / 100,
        "memory_available_gb": stats['system_resources']['memory_available_gb'],
        "gpu_memory_gb": stats['system_resources']['gpu'].get('memory_total', 0),
        "models_loaded": stats['models_loaded'],
        "tasks_processed": stats['processing_stats']['tasks_processed']
    }

@app.post("/process")
async def process_task(request: ProcessingRequest, token: str = Depends(verify_token)):
    """Main processing endpoint with authentication"""
    try:
        task_type = request.task_type
        payload = request.payload
        
        result = None
        
        if task_type == "text_generation":
            result = await processor.process_text(
                payload.get('text', ''),
                payload.get('model', 'default'),
                payload.get('parameters', {})
            )
        
        elif task_type == "image_generation":
            result = await processor.generate_image(
                payload.get('prompt', ''),
                payload.get('style'),
                payload.get('size', '512x512'),
                steps=payload.get('steps', 25),
                guidance_scale=payload.get('guidance_scale', 7.5)
            )
        
        elif task_type == "embeddings":
            result = await processor.generate_embeddings(
                payload.get('texts', []),
                payload.get('model', 'sentence-transformers')
            )
        
        else:
            raise ValueError(f"Unknown task type: {task_type}")
        
        return {
            'task_id': request.task_id,
            'status': 'completed',
            'result': result,
            'session_id': processor.session_id
        }
        
    except Exception as e:
        logger.error(f"Processing error: {e}")
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/text", dependencies=[Depends(verify_token)])
async def generate_text(request: TextRequest):
    """Direct text generation endpoint"""
    try:
        result = await processor.process_text(
            request.text,
            request.model,
            request.parameters
        )
        return {"success": True, "result": result}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/image", dependencies=[Depends(verify_token)])
async def generate_image(request: ImageRequest):
    """Direct image generation endpoint"""
    try:
        result = await processor.generate_image(
            request.prompt,
            request.style,
            request.size,
            steps=request.steps,
            guidance_scale=request.guidance_scale
        )
        return {"success": True, "result": result}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/embeddings", dependencies=[Depends(verify_token)])
async def generate_embeddings(request: EmbeddingRequest):
    """Direct embeddings generation endpoint"""
    try:
        result = await processor.generate_embeddings(
            request.texts,
            request.model
        )
        return {"success": True, "result": result}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/stats", dependencies=[Depends(verify_token)])
async def get_stats():
    """Get detailed system statistics"""
    return processor.get_system_stats()

def start_server(port=8080):
    """Start the FastAPI server"""
    uvicorn.run(app, host="0.0.0.0", port=port, log_level="info")

print("🌐 Sun Colab FastAPI server configured with authentication")
print(f"   API Key: {processor.api_key[:20] if processor.api_key else 'Not set'}...")

## 🚀 Deploy with ngrok

In [ ]:
from pyngrok import ngrok
import threading
import time
import requests

# Start server in background
def run_server():
    start_server(port=8080)

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Wait for server to start
print("⏳ Starting server...")
time.sleep(5)

# Create ngrok tunnel
try:
    # Kill any existing tunnels
    ngrok.kill()
    
    # Create new tunnel
    public_url = ngrok.connect(8080)
    
    print(f"\n🌟 Sun Colab Server Successfully Deployed!")
    print(f"\n📋 Server Details:")
    print(f"   Public URL: {public_url}")
    print(f"   Session ID: {processor.session_id}")
    print(f"   API Key: {processor.api_key[:20]}...")
    
    print(f"\n🔧 Flask Integration:")
    print(f"   Add to your .env file:")
    print(f"   COLAB_PROCESSING_URLS={public_url}")
    print(f"   SUN_COLAB_KEY={processor.api_key}")
    
    print(f"\n🧪 Test Commands:")
    print(f"   curl -X GET {public_url}/health")
    print(f"   curl -X GET {public_url}/ ")
    
    # Test the server
    try:
        health_response = requests.get(f"{public_url}/health", timeout=10)
        if health_response.status_code == 200:
            health_data = health_response.json()
            print(f"\n✅ Server Health Check Passed:")
            print(f"   Status: {health_data['status']}")
            print(f"   GPU Memory: {health_data.get('gpu_memory_gb', 0):.1f} GB")
            print(f"   Models Ready: {sum(health_data['models_loaded'].values())}/3")
        else:
            print(f"\n❌ Health check failed: {health_response.status_code}")
    except Exception as e:
        print(f"\n⚠️  Health check error: {e}")
    
    print(f"\n🔒 Security Features:")
    print(f"   ✅ Bearer token authentication")
    print(f"   ✅ Colab secrets integration")
    print(f"   ✅ Request validation")
    
except Exception as e:
    print(f"\n❌ Failed to create ngrok tunnel: {e}")
    print("\n💡 Troubleshooting:")
    print("   1. Make sure ngrok is installed")
    print("   2. Check if port 8080 is available")
    print("   3. Restart the runtime if needed")

## 🧪 Test Processing Endpoints

In [ ]:
import requests
import json
import base64
from IPython.display import Image, display

print("🧪 Testing Sun Colab Processing Endpoints\n")

# Get the API endpoint and key
api_url = str(public_url)
api_key = processor.api_key
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json'
}

# Test 1: Text Generation
print("1️⃣ Testing Text Generation...")
try:
    text_data = {
        "text": "The future of artificial intelligence is",
        "parameters": {
            "max_length": 100,
            "temperature": 0.8
        }
    }
    
    response = requests.post(f"{api_url}/text", json=text_data, headers=headers, timeout=30)
    
    if response.status_code == 200:
        result = response.json()
        generated = result['result']['new_text'][:100]
        processing_time = result['result']['processing_time']
        print(f"   ✅ Generated: '{generated}...'")
        print(f"   ⏱️  Processing time: {processing_time:.2f}s")
    else:
        print(f"   ❌ Error: {response.status_code} - {response.text}")
except Exception as e:
    print(f"   ❌ Exception: {e}")

print("\n" + "="*50 + "\n")

# Test 2: Image Generation
print("2️⃣ Testing Image Generation...")
try:
    image_data = {
        "prompt": "a beautiful sunset over mountains, digital art",
        "style": "artistic",
        "size": "512x512",
        "steps": 20
    }
    
    response = requests.post(f"{api_url}/image", json=image_data, headers=headers, timeout=60)
    
    if response.status_code == 200:
        result = response.json()
        processing_time = result['result']['processing_time']
        image_b64 = result['result']['image']
        
        print(f"   ✅ Image generated successfully")
        print(f"   ⏱️  Processing time: {processing_time:.2f}s")
        print(f"   📐 Size: {result['result']['size']}")
        print(f"   🎨 Style: {result['result']['style']}")
        
        # Display the image
        try:
            image_data = base64.b64decode(image_b64)
            display(Image(data=image_data, width=300))
        except:
            print("   ℹ️  Image generated but display failed (normal in some environments)")
    else:
        print(f"   ❌ Error: {response.status_code} - {response.text}")
except Exception as e:
    print(f"   ❌ Exception: {e}")

print("\n" + "="*50 + "\n")

# Test 3: Embeddings
print("3️⃣ Testing Embeddings Generation...")
try:
    embedding_data = {
        "texts": [
            "Machine learning is revolutionizing technology",
            "Artificial intelligence will transform industries",
            "Deep learning models are becoming more powerful"
        ]
    }
    
    response = requests.post(f"{api_url}/embeddings", json=embedding_data, headers=headers, timeout=30)
    
    if response.status_code == 200:
        result = response.json()
        embeddings = result['result']['embeddings']
        shape = result['result']['shape']
        processing_time = result['result']['processing_time']
        
        print(f"   ✅ Embeddings generated successfully")
        print(f"   📊 Shape: {shape}")
        print(f"   ⏱️  Processing time: {processing_time:.2f}s")
        
        # Calculate similarity between first two texts
        import numpy as np
        emb1 = np.array(embeddings[0])
        emb2 = np.array(embeddings[1])
        similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
        print(f"   🔗 Similarity (text 1-2): {similarity:.3f}")
    else:
        print(f"   ❌ Error: {response.status_code} - {response.text}")
except Exception as e:
    print(f"   ❌ Exception: {e}")

print("\n" + "="*50 + "\n")

# Test 4: System Stats
print("4️⃣ Testing System Statistics...")
try:
    response = requests.get(f"{api_url}/stats", headers=headers, timeout=10)
    
    if response.status_code == 200:
        stats = response.json()
        
        print(f"   ✅ Stats retrieved successfully")
        print(f"   🖥️  Session ID: {stats['session_info']['session_id']}")
        print(f"   ⏰ Uptime: {stats['session_info']['uptime_minutes']:.1f} minutes")
        print(f"   📊 Tasks processed: {stats['processing_stats']['tasks_processed']}")
        print(f"   🔧 Models loaded: {sum(stats['models_loaded'].values())}/3")
        print(f"   💾 Memory usage: {stats['system_resources']['memory_percent']:.1f}%")
        
        if stats['system_resources']['gpu']:
            gpu = stats['system_resources']['gpu']
            print(f"   🎮 GPU: {gpu.get('name', 'Unknown')}")
            print(f"   🎮 GPU Memory: {gpu.get('memory_allocated', 0):.1f}/{gpu.get('memory_total', 0):.1f} GB")
    else:
        print(f"   ❌ Error: {response.status_code} - {response.text}")
except Exception as e:
    print(f"   ❌ Exception: {e}")

print("\n🎉 All tests completed!")
print(f"\n📋 Integration Summary:")
print(f"   Server URL: {api_url}")
print(f"   Session ID: {processor.session_id}")
print(f"   Authentication: ✅ Active")
print(f"   GPU Processing: ✅ Available")

## 🔄 Keep Server Running

In [ ]:
import time
import requests
from datetime import datetime

print("🌟 Sun Colab Server is running!")
print(f"\n📋 Connection Details:")
print(f"   URL: {public_url}")
print(f"   API Key: {processor.api_key}")
print(f"   Session: {processor.session_id}")

print(f"\n🔧 Add to your Flask .env:")
print(f"   COLAB_PROCESSING_URLS={public_url}")
print(f"   SUN_COLAB_KEY={processor.api_key}")

print(f"\n⚠️  Keep this cell running to maintain the server!")
print(f"\n📊 Real-time monitoring:")

# Monitoring loop
start_time = datetime.now()
check_count = 0

try:
    while True:
        time.sleep(30)  # Check every 30 seconds
        check_count += 1
        
        try:
            # Health check
            health = requests.get(f"{public_url}/health", timeout=5)
            
            if health.status_code == 200:
                data = health.json()
                uptime = (datetime.now() - start_time).total_seconds() / 60
                
                status_line = (
                    f"\r[{datetime.now().strftime('%H:%M:%S')}] "
                    f"✅ Healthy | "
                    f"Uptime: {uptime:.1f}m | "
                    f"Tasks: {data.get('tasks_processed', 0)} | "
                    f"Memory: {data.get('memory_available_gb', 0):.1f}GB | "
                    f"Checks: {check_count}"
                )
                print(status_line, end='', flush=True)
                
                # Detailed status every 10 checks (5 minutes)
                if check_count % 10 == 0:
                    print(f"\n\n📊 Status Update (Check #{check_count}):")
                    print(f"   Uptime: {uptime:.1f} minutes")
                    print(f"   Tasks Processed: {data.get('tasks_processed', 0)}")
                    print(f"   GPU Memory: {data.get('gpu_memory_gb', 0):.1f} GB")
                    print(f"   Models Loaded: {sum(data.get('models_loaded', {}).values())}/3")
                    print(f"   Session ID: {data.get('session_id', 'Unknown')}")
                    print("\n📡 Continuing monitoring...")
            else:
                print(f"\r❌ Health check failed: {health.status_code}", end='', flush=True)
                
        except Exception as e:
            print(f"\r⚠️  Connection error: {str(e)[:50]}", end='', flush=True)
            
except KeyboardInterrupt:
    print(f"\n\n🛑 Monitoring stopped by user")
    final_uptime = (datetime.now() - start_time).total_seconds() / 60
    print(f"📊 Final Statistics:")
    print(f"   Total uptime: {final_uptime:.1f} minutes")
    print(f"   Health checks: {check_count}")
    print(f"   Session ID: {processor.session_id}")
except Exception as e:
    print(f"\n\n❌ Monitoring error: {e}")
    print("Server may still be running. Check manually with health endpoint.")